# import

In [1]:
## import functions from .py utils file: 
from Utilities.preprocessing import *
import os
from IPython.display import display, HTML
from tqdm import tqdm 
from datetime import datetime


# paths

In [9]:
InputPath = r"Z:\projects\sequence_squad\revision_data\lars_recordings\bpod_data\\"
# InputPath = r"D:\EPHYS_BPOD_data_example\\"

out_path_base = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"

Replace = True



# set paths

In [40]:

# find main paths for the animals
main_paths = []
Animal_ID = []
Animal_save_paths = []
Animal_bpod_save_paths = []
Animal_ephys_dates = []

# for each animal in the bpod dump path 
for mouse in os.listdir(InputPath):
    if not mouse == 'FakeSubject':
        Animal_ID += [mouse]
        main_paths+=[os.path.join(InputPath,mouse,r'Sequence_Automated','Session Data//')]
        save_paths = []
        # find dates that match an ephys recording in teh organised folder
        for animal_file in os.listdir(out_path_base):
            # for seq00 animals
            if mouse.lower() in animal_file:
                recordings = os.listdir(os.path.join(out_path_base,animal_file))
                # Sort by date
                sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                save_paths = list(os.path.join(out_path_base,animal_file,recording) for recording in sorted_recordings)
                break
                ## extract date from recordings
           
            elif 'ap5' in mouse.lower():
                if 'lr' in mouse.lower():
                    recordings = os.listdir(os.path.join(out_path_base,'ap5lr_implant1'))
                    # Sort by date
                    sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                    ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                    save_paths = list(os.path.join(out_path_base,'ap5lr_implant1',recording) for recording in sorted_recordings)
                    break
                elif 'L' in mouse:
                    recordings = os.listdir(os.path.join(out_path_base,'ap5L_implant1'))
                    # Sort by date
                    sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                    ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                    save_paths = list(os.path.join(out_path_base,'ap5L_implant1',recording) for recording in sorted_recordings)
                    break
                elif 'R' in mouse:
                    recordings = os.listdir(os.path.join(out_path_base,'ap5R_implant1'))
                    # Sort by date
                    sorted_recordings = sorted(recordings, key=lambda x: datetime.strptime(x.split('_')[1], '%d-%m-%Y'))
                    ephys_dates = list(recording.split('_')[-1] for recording in sorted_recordings)
                    save_paths = list(os.path.join(out_path_base,'ap5R_implant1',recording) for recording in sorted_recordings)
                    break

        if len(save_paths) > 0: # ignore for files that are not in ephys organised folder
            # create full save paths 
            full_save_paths = []
            for save_path in save_paths:
                new_path = os.path.join(save_path,r'behav_sync/2_task/')
                if not os.path.isdir(new_path):
                    print('making path')
                    print(new_path)
                    os.makedirs(new_path)
                full_save_paths+=[new_path]
            Animal_bpod_save_paths += [full_save_paths]
            Animal_save_paths += [save_paths]
            Animal_ephys_dates += [ephys_dates]
        
      


making path
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq006_implant1\recording11_28-11-2024\behav_sync/2_task/


In [41]:
        
#### MAIN ####
for Animal_index, CurrentAnimal in enumerate(Animal_ID):
    if Animal_index > 2:
            
        display(HTML(f"<b>Processing data for: {CurrentAnimal}</b>"))
        
        #Load Behavioural data:
        Behav_data, Sessions,Path,FileDates = Import_EPHYS_Bpod_DataFiles(main_paths[Animal_index],Animal_ephys_dates[Animal_index])
        
        Processed = ('')
        Skipped = ('')


        for Session in tqdm(range(0, Sessions)):

            print(Session)

            filedate = FileDates[Session] + '_' + str(Behav_data[Session]['__header__'])[-25:-22]

            if Session < 10:
                Save_path = (Animal_save_paths[Animal_index][Session] + r'\\'+ r'\behav_sync\2_task\Preprocessed' + r'\\0' + str(Session) + '_' + filedate)
            else:
                Save_path = (Animal_save_paths[Animal_index][Session] + r'\\' + r'\behav_sync\2_task\Preprocessed' + r'\\' + str(Session) + '_' + filedate)
                
                
            print(Save_path)

            # see if dir exists already then use bool logic to detemrine if it needs to be processed or not
            Process = check_and_create_directory(Save_path, Replace)

            if Process == True:
            
                #Convert to python friendly format:
                convert_nested_structs(Behav_data[Session])

            #    # Extract GUI info
                Trial_settings = todict(Behav_data[Session]['SessionData']['TrialSettings'][0])
                FinalRewardAmount = []
                for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                    TLevel = item
                    FinalRewardAmount = FinalRewardAmount + [Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][4]]

                # save out training levels on their own
                filename = 'Preprocessed_TrainingLevels' 
                with open(Save_path + r'\\'+ filename, 'wb') as fp:
                    pickle.dump(Behav_data[Session]['SessionData']['SessionVariables']['TLevel'], fp)

                # save out led intensites and reward amounts on their own:
                LED_Intensities = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port1'],
                                                'Port2':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port2'],
                                                'Port3':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port3'],
                                                'Port4':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port4'],
                                                'Port5':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port5']})
                LED_Intensities.to_csv(Save_path + '/Preprocessed_LED_Intensities.csv')
                RewardAmounts = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port1'],
                                            'Port2':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port2'],
                                            'Port3':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port3'],
                                            'Port4':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port4'],
                                            'Port5':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port5']})
                RewardAmounts.to_csv(Save_path + '/Preprocessed_RewardAmounts.csv')


                #Extract PortIn times for each port and check for errors (inside this function):
                All_PortIn_Times,All_PortOut_Times,All_Port_references = extract_poke_times(Behav_data[Session])

                #remove nans (times when part [in or out poke] of the event was dropped for some reason by bpod)
                All_PortIn_Times_fixed,All_PortOut_Times_fixed ,All_Port_references_fixed = remove_dropped_in_events(All_PortIn_Times,All_PortOut_Times,All_Port_references)
                
                # Resort these in time:
                All_PortIn_Times_sorted,All_PortOut_Times_sorted,All_Port_references_sorted = time_sort(All_PortIn_Times,All_PortOut_Times,All_Port_references)

                #extract reward times:
                Reward_ts = extract_reward_times(Behav_data[Session])
                
                # find reward inds and align rewarded ts to poke events:
                Rewarded_event_inds = find_reward_inds(All_PortIn_Times_sorted,All_Port_references_sorted,Reward_ts)
                Reward_ts = np.asarray(Reward_ts)
                Reward_ts = Reward_ts[np.logical_not(np.isnan(Reward_ts))]
                Reward_ts = list(Reward_ts)
                Reward_ts_aligned = align_trigger_to_index(Reward_ts,Rewarded_event_inds,All_Port_references_sorted)

                #extract trial start time stamps
                Trial_start_ts = extract_trial_timestamps(Behav_data[Session])

                #extract trial end times:
                Trial_end_ts = extract_trial_end_times(Behav_data[Session])

                #determine trial IDs
                trial_id = determine_trial_id(All_PortIn_Times_sorted,Trial_end_ts)

                # align trial start and end times to poke events
                trialstart_index = find_trialstart_index(trial_id)
                trial_start_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_start_ts)
                trial_end_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_end_ts)

                #determine LED and reward states for each trial and align them to trials:
                IntermediateRewards = []
                LED_intensities = []
                for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                    TLevel = item
                    IntermediateRewards = IntermediateRewards + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][0:4])]
                    LED_intensities = LED_intensities + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][6:10])]
                aligned_LED_intensities = align_trial_start_end_timestamps(trial_id,trialstart_index,LED_intensities)
                aligned_IntermediateRewards = align_trial_start_end_timestamps(trial_id,trialstart_index,IntermediateRewards)


                Trial_start_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)
                Trial_end_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)
                First_poke_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)

                ## align Training level for each trial:
                Training_Levels = align_opto_trials_to_dataframe(trial_id,Behav_data[Session]['SessionData']['SessionVariables']['TLevel'])

                # make portin dataframe:
                PortIn_df = pd.DataFrame(
                    {'Trial_id' : trial_id,
                    'Trial_Start' : trial_start_ts_aligned,
                    'Port': All_Port_references_sorted,
                    'PokeIn_Time': All_PortIn_Times_sorted,
                    'PokeOut_Time': All_PortOut_Times_sorted,
                    'Reward_Times': Reward_ts_aligned,
                    'Trial_End' : trial_end_ts_aligned,
                    'Port 2,3,4,5 LED intensities': aligned_LED_intensities,
                    'Port 1,2,3,4 RewardAmount':aligned_IntermediateRewards,
                    'BACK_Trial_Start_Camera_Time' : Trial_start_Camera_Ts_aligned,
                    'BACK_Trial_End_Camera_Time' : Trial_end_Camera_Ts_aligned,
                    'BACK_First_poke_Camera_Time' : First_poke_Camera_Ts_aligned,
                    'TrainingLevel': Training_Levels})

                #Save Data
                PortIn_df.to_csv(Save_path +'/PreProcessed_RawPokeData.csv')


SEQ006_Sequence_Automated_20241115_111239.mat
SEQ006_Sequence_Automated_20241116_134904.mat
SEQ006_Sequence_Automated_20241118_145435.mat
SEQ006_Sequence_Automated_20241119_112407.mat
SEQ006_Sequence_Automated_20241120_105129.mat
SEQ006_Sequence_Automated_20241121_105847.mat


  0%|          | 0/6 [00:00<?, ?it/s]

0
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq006_implant1\recording1_15-11-2024\\\behav_sync\2_task\Preprocessed\\00_20241115_111239_Fri


 17%|█▋        | 1/6 [00:20<01:41, 20.38s/it]

1
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq006_implant1\recording2_16-11-2024\\\behav_sync\2_task\Preprocessed\\01_20241116_134904_Sat


...

2
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq008_implant1\recording3_13-11-2024\\\behav_sync\2_task\Preprocessed\\02_20241113_104324_Wed


 75%|███████▌  | 3/4 [01:27<00:28, 28.97s/it]

3
Z:\projects\sequence_squad\revision_data\organised_data\animals\\seq008_implant1\recording4_15-11-2024\\\behav_sync\2_task\Preprocessed\\03_20241115_153541_Fri


100%|██████████| 4/4 [01:53<00:00, 28.27s/it]


IndexError: list index out of range